# Setup

In [ ]:
%%capture
!pip install -qU langchain
!pip install -qU langchain-google-genai
!pip install -qU langchain-huggingface
!pip install -qU langchain-qdrant
!pip install -qU langchain-community
!pip install -qU langgraph
!pip install fastembed
!pip install datasets
!pip install -U "fsspec[http]==2024.10.0"
!pip install fastapi 
!pip install uvicorn

In [2]:
%%capture
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import torch
from tqdm import tqdm

from IPython.display import display

from sklearn.utils import resample
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer

from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

# Components

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-001")

In [ ]:
os.environ["LANGSMITH_TRACING_V2"] = "true"
os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGSMITH_API_KEY"] = userdata.get('langsmith_api_key')
os.environ["LANGSMITH_PROJECT"] = "DSDE-Project"

# Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! ls 'drive/MyDrive/Data Sci 2025/Final_Project/Final Project/'

bangkok_traffy.csv  df_for_ds.csv  handred_thousand.csv
DataEng.ipynb	    full_col.csv   minidf_notype.csv


In [ ]:
dataset_path = 'drive/MyDrive/Data Sci 2025/Final_Project/Final Project/'
ds = pd.read_csv(dataset_path + "handred_thousand.csv")

# Embedding Model

In [ ]:
"""
"KanisornPutta/CeltaVigoBert"
"clicknext/phayathaibert"
"KanisornPutta/TrentIsNotFuckingLeavingBERT"
"""

'\n"KanisornPutta/CeltaVigoBert"\n"clicknext/phayathaibert"\n"KanisornPutta/TrentIsNotFuckingLeavingBERT"\n'

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

model_kwargs = {'trust_remote_code': True}
embeddings = HuggingFaceEmbeddings(model_name="KanisornPutta/TrentIsNotFuckingLeavingBERT",model_kwargs=model_kwargs)

In [ ]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder

model = HuggingFaceCrossEncoder(model_name="Pongsasit/mod-th-cross-encoder-minilm")

## Data Preparation

In [ ]:
ds.head(3)

,ticket_id,type,organization,comment,photo,photo_after,coords,address,subdistrict,district,province,timestamp,state,star,count_reopen,last_activity,type_str,type_set
0,2024-HDE8AM,"{เสียงรบกวน,ถนน}",กองโครงการสายอากาศเป็นสายใต้ดิน2 ฝ่ายบริหารโคร...,แผ่นเหล็กปิดถนนขั่วคราวปิดไม่เรียบร้อย เกิดเสี...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,"100.65690,13.76710",968 ถนน รามคำแหง หัวหมาก เขตบางกะปิ กรุงเทพมหา...,หัวหมาก,บางกะปิ,กรุงเทพมหานคร,2024-07-06 01:26:42.908463+00,เสร็จสิ้น,NaN,0,2024-11-06 02:00:11.876998+00,"เสียงรบกวน,ถนน","{'ถนน', 'เสียงรบกวน'}"
1,2023-E4NQUH,{ถนน},"เขตประเวศ,ฝ่ายโยธา เขตประเวศ",https://youtu.be/7ly_Zgk-8Co\n\nดูจบแล้วน่าสนใ...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,"100.69469,13.71739",33 เฉลิมพระเกีรยติ ร.9 ซอย 81 แขวงประเวศ เขต ป...,ประเวศ,ประเวศ,กรุงเทพมหานคร,2023-05-02 14:35:58.588305+00,เสร็จสิ้น,NaN,0,2023-07-14 07:33:28.60558+00,ถนน,{'ถนน'}
2,2024-VR89N3,{ถนน},"เขตบางซื่อ,สำนักการโยธา กทม.,สำนักงานวิศวกรรมท...",พบปัญหายางมะตอยที่ราดบนสะพานหลุดล่อนทำให้เกิดร...,https://storage.googleapis.com/traffy_public_b...,NaN,"100.53901,13.83039",51 ถ. รัชดาภิเษก แขวง วงศ์สว่าง เขตบางซื่อ กรุ...,วงศ์สว่าง,บางซื่อ,กรุงเทพมหานคร,2024-05-25 10:23:11.112647+00,กำลังดำเนินการ,NaN,0,2024-05-26 04:32:04.804649+00,ถนน,{'ถนน'}


In [ ]:
ds['comment'][4]

'ทางเท้า หน้าร้านอาหาร ซอยสุขุมวิท23 สกปรก โสโครกมากก ร้านค้าเทสิ่งปฏิกูลลงท่อ ลงทางเท้า ร้องเรียนไปหลายครั้งมากๆๆๆ จนรำคาญการทำงานของเจ้าหน้าที่ ที่ไม่ทำ***อะไรเลย ช่วยไปทำความสะอาด และปรับร้านค้าด้วย ฟรือต้องให้ร้องเรียนแบบนี้ทุกวัน ทำงานบ้างเถอะ'

In [ ]:
ds['address'][4]

'4 7 ซ. สุขุมวิท 23 แขวงคลองเตยเหนือ เขตวัฒนา กรุงเทพมหานคร 10110 ประเทศไทย'

In [ ]:
ds['photo'][0]

'https://storage.googleapis.com/traffy_public_bucket/attachment/2024-07/198951fea260fcf8e727e50942bd35519fc3fa6a.jpg'

In [ ]:
import re

def clean_comment(comment):

    if type(comment) != str :
      return ""

    # comment = ''.join([' ' if c.isdigit() else c for c in comment])
    comment = ''.join([c for c in comment if c.isalpha() or c.isspace() or '\u0E00' <= c <= '\u0E7F'])
    comment = comment.replace('\n', ' ')
    comment = comment.replace('\r', ' ')
    comment = re.sub(' +', ' ', comment)  # replaces multiple spaces with one

    return comment

In [ ]:
def seperate_text(text):
    if type(text) != str :
      return ""
    return text.replace(",", ", ")

In [ ]:
ds['type_str'] = ds['type_str'].apply(seperate_text)

In [ ]:
ds['type_str'][0]

'เสียงรบกวน, ถนน'

In [ ]:
ds.shape

(100000, 18)

In [ ]:
ds.dropna(subset=['comment'], inplace=True)

In [ ]:
ds.shape

(99035, 18)

In [ ]:
ds['context'] = ds.apply(
    lambda row: f'ชนิด: {seperate_text(row["type_str"])} : "{clean_comment(row["comment"])}" ที่อยู่ "{clean_comment(row["address"])}"',
    axis=1
)

In [ ]:
ds.loc[4]

,4
ticket_id,2024-E7E2VX
type,"{ร้องเรียน,ทางเท้า}"
organization,"เขตวัฒนา,ฝ่ายเทศกิจ เขตวัฒนา,ฝ่ายสิ่งแวดล้อมฯ ..."
comment,ทางเท้า หน้าร้านอาหาร ซอยสุขุมวิท23 สกปรก โสโค...
photo,https://storage.googleapis.com/traffy_public_b...
photo_after,https://storage.googleapis.com/traffy_public_b...
coords,"100.56279,13.73644"
address,4 7 ซ. สุขุมวิท 23 แขวงคลองเตยเหนือ เขตวัฒนา ก...
subdistrict,คลองเตยเหนือ
district,วัฒนา


In [ ]:
ds['context'][4]

'ชนิด: ร้องเรียน,  ทางเท้า : "ทางเท้า หน้าร้านอาหาร ซอยสุขุมวิท สกปรก โสโครกมากก ร้านค้าเทสิ่งปฏิกูลลงท่อ ลงทางเท้า ร้องเรียนไปหลายครั้งมากๆๆๆ จนรำคาญการทำงานของเจ้าหน้าที่ ที่ไม่ทำอะไรเลย ช่วยไปทำความสะอาด และปรับร้านค้าด้วย ฟรือต้องให้ร้องเรียนแบบนี้ทุกวัน ทำงานบ้างเถอะ" ที่อยู่ " ซ สุขุมวิท แขวงคลองเตยเหนือ เขตวัฒนา กรุงเทพมหานคร ประเทศไทย"'

In [ ]:
from langchain_core.documents import Document

In [ ]:
all_docs = []

for index, row in ds.iterrows(): # Use iterrows() for index-safe iteration
    context = row["context"]
    problem_type = row["type_str"]  # Get values directly from the row
    address = row["address"]
    ticket_id = row["ticket_id"]
    comment = row["comment"]

    if context is not None:
        doc = Document(
            page_content=str(context),
            metadata={
                "problem_type": problem_type,
                "address": address,
                "ticket_id": ticket_id
            }
        )
        all_docs.append(doc)

print(len(all_docs))

99035


In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,  # chunk size (characters)
    chunk_overlap=200,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
)
all_splits = text_splitter.split_documents(all_docs)

print(f"Split wiki abstract into {len(all_splits)} sub-documents.")

Split wiki abstract into 99417 sub-documents.


In [ ]:
all_splits[0]

Document(metadata={'problem_type': 'เสียงรบกวน, ถนน', 'address': '968 ถนน รามคำแหง หัวหมาก เขตบางกะปิ กรุงเทพมหานคร 10240 ประเทศไทย', 'ticket_id': '2024-HDE8AM', 'start_index': 0}, page_content='ชนิด: เสียงรบกวน,  ถนน : "แผ่นเหล็กปิดถนนขั่วคราวปิดไม่เรียบร้อย เกิดเสียงดังทุกครั้งที่รถวิ่งผ่าน เป็นอันคราบต่อผู้ใช้ถนน" ที่อยู่ " ถนน รามคำแหง หัวหมาก เขตบางกะปิ กรุงเทพมหานคร ประเทศไทย"')

## Vector Database
Embed your documents in a vector database that supports hybrid search. Also set the retrieval mode to hybrid search.

We will use `QdrantVectorStore` [Learn more here](https://python.langchain.com/api_reference/qdrant/qdrant/langchain_qdrant.qdrant.QdrantVectorStore.html#langchain_qdrant.qdrant.QdrantVectorStore). (You can use any vector DB that can do hybrid search)

In [ ]:
from tqdm import tqdm

In [ ]:
from langchain_qdrant import FastEmbedSparse, QdrantVectorStore, RetrievalMode
from qdrant_client import QdrantClient, models
from qdrant_client.http.models import Distance, SparseVectorParams, VectorParams

sparse_embeddings = FastEmbedSparse(model_name="Qdrant/bm25")

# Create a Qdrant client for local storage
# client = QdrantClient(":memory:")
client = QdrantClient(path="./qdrant_storage")

# Create a collection with both dense and sparse vectors
client.create_collection(
    collection_name="my_documents",
    vectors_config={"dense": VectorParams(size=768, distance=Distance.COSINE)},
    sparse_vectors_config={
        "sparse": SparseVectorParams(index=models.SparseIndexParams(on_disk=False))
    },
)

qdrant = QdrantVectorStore(
    client=client,
    collection_name="my_documents",
    embedding=embeddings,
    sparse_embedding=sparse_embeddings,
    retrieval_mode=RetrievalMode.HYBRID,
    vector_name="dense",
    sparse_vector_name="sparse",
)

print("Adding documents to Qdrant...")
for doc in tqdm(all_splits, desc="Uploading documents"):
    qdrant.add_documents(documents=[doc])  # Process one document at a time

query = "น้ำท่วม"
found_docs = qdrant.similarity_search(query)
found_docs

### Download vector Database

In [ ]:
!zip -r qdrant_storage.zip qdrant_storage

  adding: qdrant_storage/ (stored 0%)
  adding: qdrant_storage/.lock (stored 0%)
  adding: qdrant_storage/collection/ (stored 0%)
  adding: qdrant_storage/collection/my_documents/ (stored 0%)
  adding: qdrant_storage/collection/my_documents/storage.sqlite (deflated 55%)
  adding: qdrant_storage/meta.json (deflated 59%)


In [ ]:
from google.colab import files
files.download("qdrant_storage.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Retrievers


In [ ]:
retriever = qdrant.as_retriever(search_kwargs={"k": 10})

In [ ]:
reranker = CrossEncoderReranker(model=model, top_n=10)
reranked_retriever = ContextualCompressionRetriever(
    base_compressor=reranker , base_retriever=retriever
)

Take a subset of the dataset to evaluate the MRR of the retrievers.

In [ ]:
#sample = ds.take(1000)

In [ ]:
test_query = "เขตจตุจักรมีปัญหาเกี่ยวกับอะไรมากที่สุด?"

In [ ]:
test_docs = retriever.get_relevant_documents(test_query)
print(f'query : {test_query}')
print('-'*30)
for doc in test_docs[:10] :
  print(f'- {doc.page_content}')

<ipython-input-36-834d686a6aae>:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  test_docs = retriever.get_relevant_documents(test_query)


query : เขตจตุจักรมีปัญหาเกี่ยวกับอะไรมากที่สุด?
------------------------------
- การเป็นพยาน หรือการให้ข้อมูลนั้น ศรท ศรท citydata" ที่อยู่ "สำนักงานเขตบางแค ซอย กาญจนาภิเษก แยก แขวงบางแค เขตบางแค กรุงเทพมหานคร ซอย กาญจนาภิเษก แยก แขวงบางแค เขตบางแค กรุงเทพมหานคร ประเทศไทย"
- ชนิด: ร้องเรียน : "ขอเรียนแจ้งปัญหาเกี่ยวกับการทำงานของผู้รับเหมา น่าจะเป็น TOT มีการทำงานที่ไม่เรียบร้อย ส่งผลให้ฝรั่งนักท่องเที่ยวหยุดถ่ายรูปหลายครั้ง ปัญหาเมือง" ที่อยู่ "Opposite Baan Klang Muang Lat Phrao แขวง จันทรเกษม เขตจตุจักร กรุงเทพมหานคร ประเทศไทย"
- หรือการให้ข้อมูลนั้น ศรท ศรท citydata" ที่อยู่ "สำนักงานเขตพญาไท ซอย อารีย์ แขวง พญาไท เขตพญาไท กรุงเทพมหานคร ซอย อารีย์ แขวง พญาไท เขตพญาไท กรุงเทพมหานคร ประเทศไทย"
- ชนิด: ถนน : "ตลาดนัดจตุจักร เรียนผู้ว่ากทม ขอให้ตรวจสอบกรณีวันที่กค ณห้องประชุมตลาดนัดจตุจักร มีชมรมผู้ค้ากลุ่มนึงเข้ามาส่งจดหมายติดตาม คำสั่งจากปลัดกรุงเทพมหานคร ที่สั่งให้มีการทดลองจอดรถในตลาดนัดจตุจักรทั้งวันเป็นเวลาเดือนกค ถึง กย ว่าเป็นคำสั่งที่ชอบธรรมหรือไม่ และเอากฎระเบียบข้อไหนมารอง

In [ ]:
test_docs_reranked = reranked_retriever.get_relevant_documents(test_query)
print(f'query : {test_query}')
print('-'*30)
for doc in test_docs_reranked[:10] :
  print(f'- {doc.page_content}')

query : เขตจตุจักรมีปัญหาเกี่ยวกับอะไรมากที่สุด?
------------------------------
- ชนิด: ร้องเรียน : "ขอเรียนแจ้งปัญหาเกี่ยวกับการทำงานของผู้รับเหมา น่าจะเป็น TOT มีการทำงานที่ไม่เรียบร้อย ส่งผลให้ฝรั่งนักท่องเที่ยวหยุดถ่ายรูปหลายครั้ง ปัญหาเมือง" ที่อยู่ "Opposite Baan Klang Muang Lat Phrao แขวง จันทรเกษม เขตจตุจักร กรุงเทพมหานคร ประเทศไทย"
- ชนิด: ถนน : "ตลาดนัดจตุจักร เรียนผู้ว่ากทม ขอให้ตรวจสอบกรณีวันที่กค ณห้องประชุมตลาดนัดจตุจักร มีชมรมผู้ค้ากลุ่มนึงเข้ามาส่งจดหมายติดตาม คำสั่งจากปลัดกรุงเทพมหานคร ที่สั่งให้มีการทดลองจอดรถในตลาดนัดจตุจักรทั้งวันเป็นเวลาเดือนกค ถึง กย ว่าเป็นคำสั่งที่ชอบธรรมหรือไม่ และเอากฎระเบียบข้อไหนมารองรับ โดยผู้ค้ากลุ่มนี้อ้างว่าถ้าให้รถจอดริมถนนรอบตลาดนัดแล้วจะทำให้ลูกค้าเดินเข้าไปด้านในทำให้ด้านในขายดีขึ้น เป็นการพิจารณาที่รับฟังความข้างเดียว เป็นการแก้ปัญหาที่ไม่ตรงจุด ปัจจุบันการค้าขายรอบๆถนนในตลาดนัดจตุจักรกำลังเริ่มคึกคักหลังจากรัฐบาลมีมาตรการผ่อนปรนโรคโควิด นักท่องเที่ยวกำลังกลับมาจับจ่ายใช้สอย แต่คำสั่งดังกล่าวจะเป็นการทำลายวิถีอัตลักษณ์การเป็นตลาดนัด

In [ ]:
for doc in test_docs_reranked[:10] :
  print(f'- {doc.metadata.get("ticket_id")}')

- 2024-4LTPRV
- 2022-KL3AUW
- 2022-DU7XZE
- 2024-HEUK6W
- 4P4NKX
- 2RMUNW
- EFJXKA
- EAUC8C
- 72NZDY
- 2023-AWD87A


# Agentic RAG


In [ ]:
from langgraph.graph import MessagesState, StateGraph

graph_builder = StateGraph(MessagesState)

In [ ]:
from langchain_core.tools import tool

@tool(response_format="content")
def retrieve(query: str):
    """Retrieve information related to a query from a vector database of Traffy Fondue Dataset."""
    retrieved_docs = reranked_retriever.get_relevant_documents(query)
    serialized = "\n\n".join(
        (

            f"ticket_id: {doc.metadata.get('ticket_id')}\n"
            f"ประเภท: {doc.metadata.get('problem_type')}\n"
            f"สถานที่: {doc.metadata.get('address')}\n"
            f"รายละเอียด: {doc.page_content}"
        )
        for doc in retrieved_docs
    )
    return serialized


In [ ]:
from langchain_core.tools import tool
from langchain_core.messages import SystemMessage
from langgraph.prebuilt import ToolNode
from langchain_core.runnables import RunnableConfig

# Step 1: Generate an AIMessage that may include a tool-call to be sent.
def query_or_respond(state: MessagesState):
    """Generate tool call for retrieval or respond."""
    llm_with_tools = llm.bind_tools([retrieve])
    response = llm_with_tools.invoke(state["messages"])
    # MessagesState appends messages to state instead of overwriting
    return {"messages": [response]}


# Step 2: Execute the retrieval.
# The ToolNode is roughly analogous to:

# tools_by_name = {tool.name: tool for tool in tools}
# def tool_node(state: dict):
#     result = []
#     for tool_call in state["messages"][-1].tool_calls:
#         tool = tools_by_name[tool_call["name"]]
#         observation = tool.invoke(tool_call["args"])
#         result.append(ToolMessage(content=observation, tool_call_id=tool_call["id"]))
#     return {"messages": result}

tools = ToolNode([retrieve])


# Step 3: Generate a response using the retrieved content.
def generate(state: MessagesState):
    """Generate answer based on retrieved problem reports."""
    # Get retrieved ToolMessages
    recent_tool_messages = []
    for message in reversed(state["messages"]):
        if message.type == "tool":
            recent_tool_messages.append(message)
        else:
            break
    tool_messages = recent_tool_messages[::-1]

    # Format tool messages for structured data
    structured_entries = []
    for tool_msg in tool_messages:
        content = tool_msg.content  # This is the string returned from the tool
        structured_entries.append(content)

    docs_content = "\n\n".join(structured_entries)

    # System message prompt
    system_message_content = (
        "คุณเป็นผู้ช่วยที่เชี่ยวชาญในการตอบคำถามจากข้อมูลปัญหาที่ถูกรายงานผ่านระบบแจ้งปัญหา Traffy Fondue "
        "ข้อมูลแต่ละรายการจะประกอบด้วยประเภทของปัญหา, สถานที่ และรายละเอียดของปัญหา "
        "กรุณาตอบคำถามจากข้อมูลด้านล่าง ถ้าคุณไม่พบคำตอบที่ตรง ให้ตอบว่า 'ไม่พบข้อมูลที่เกี่ยวข้อง' และกล่าวถึงข้อมูลที่ใกล้เขียง เช่น สถานที่ใกล้เคียง หรือ ข้อมูลประเภทเดียวกัน"
        "เลือกเพียงข้อมูลที่มีความเกี่ยวข้องกับ คำถาม และ ตอบ ticket_id ที่เกี่ยวของมา หลังจบคำอธิบาย ใน format ticket_id : _id1, _id2, ..."
        "สรุปคำตอบออกมา และกล่าวถึงข้อมูลที่น่าสนใจ\n\n"
        f"{docs_content}"
    )

    conversation_messages = [
        message
        for message in state["messages"]
        if message.type in ("human", "system")
        or (message.type == "ai" and not message.tool_calls)
    ]

    prompt = [SystemMessage(system_message_content)] + conversation_messages

    # Run model
    response = llm.invoke(prompt)
    return {"messages": [response]}


In [ ]:
from langgraph.graph import END
from langgraph.prebuilt import ToolNode, tools_condition

graph_builder.add_node(query_or_respond)
graph_builder.add_node(tools)
graph_builder.add_node(generate)

graph_builder.set_entry_point("query_or_respond")
graph_builder.add_conditional_edges(
    "query_or_respond",
    tools_condition,
    {END: END, "tools": "tools"},
)
graph_builder.add_edge("tools", "generate")
graph_builder.add_edge("generate", END)

graph = graph_builder.compile()

In [ ]:
from IPython.display import Image, display

# display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()
graph = graph_builder.compile(checkpointer=memory)

# Specify an ID for the thread
config = {"configurable": {"thread_id": "abc123"}}

In [ ]:
input_message = "Hello"

for step in graph.stream(
    {"messages": [{"role": "user", "content": input_message}]},
    stream_mode="values",
    config=config,
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Hello
================================== Ai Message ==================================

Hello! How can I help you today?


In [ ]:
input_message = "What do u know"

for step in graph.stream(
    {"messages": [{"role": "user", "content": input_message}]},
    stream_mode="values",
    config=config,
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What do u know
================================== Ai Message ==================================

I have access to a vector database of Traffy Fondue Dataset, and I can retrieve information from it based on your queries. Just let me know what you're interested in!


In [ ]:
input_message = "เขตจตุจักรมีปัญหาเกี่ยวกับอะไรมากที่สุด?"

for step in graph.stream(
    {"messages": [{"role": "user", "content": input_message}]},
    stream_mode="values",
    config=config,
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

เขตจตุจักรมีปัญหาเกี่ยวกับอะไรมากที่สุด?
================================== Ai Message ==================================
Tool Calls:
  retrieve (f5ab0596-5f5a-4474-a559-735f9eb52ad1)
 Call ID: f5ab0596-5f5a-4474-a559-735f9eb52ad1
  Args:
    query: What are the most common issues reported in Chatuchak district?
================================= Tool Message =================================
Name: retrieve

ticket_id: 2024-P7MEL6
ประเภท: สายไฟ, ความปลอดภัย
สถานที่: Soi Vibhavadi Rangsit 34 Chatuchak District Office 7th Floor Vibhavadi Rangsit Road Chatuchak Khet Chatuchak Bangkok 10900 แขวงจตุจักร เขตจตุจักร กรุงเทพมหานคร 10900 ประเทศไทย
รายละเอียด: ชนิด: สายไฟ,  ความปลอดภัย : "ซอยวิภาวดีมีต้นโพธิ์ใหญ่กิ่งหนาปกคลุม ทั้งกิ่งพันสายไฟเป็นอันตรายมากกับบริเวณเเถวนี้" ที่อยู่ "Soi Vibhavadi Rangsit Chatuchak District Office th Floor Vibhavadi Rangsit Road Chatuchak Khet Chatuchak Bangkok แขวงจตุจักร เขตจตุจักร 

In [ ]:
input_message = "ขยะแถวปทุมวัน"

for step in graph.stream(
    {"messages": [
        {
          "role": "user",
          "content": input_message,
        }
    ]},
    stream_mode="values",
    config=config,
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

ขยะแถวปทุมวัน
================================== Ai Message ==================================
Tool Calls:
  retrieve (7b33ef92-5f08-4be1-a90d-6bab33484fc5)
 Call ID: 7b33ef92-5f08-4be1-a90d-6bab33484fc5
  Args:
    query: garbage issue in Pathum Wan district
================================= Tool Message =================================
Name: retrieve

ticket_id: 2022-AXX8FD
ประเภท: 
สถานที่: ติดกับร้าน Lawson 366 ร้านน้องนุ๊ก ข้างตลาดยิ่งรวย76 ตรงข้าม 108 ซอย ประชาอุทิศ 93 แขวง ทุ่งครุ เขตทุ่งครุ กรุงเทพมหานคร 10140 ประเทศไทย
รายละเอียด: ชนิด:  : "ตัวอย่างการแจ้ง รบกวนมาเก็บขยะด้วยครับ" ที่อยู่ "ติดกับร้าน Lawson ร้านน้องนุ๊ก ข้างตลาดยิ่งรวย ตรงข้าม ซอย ประชาอุทิศ แขวง ทุ่งครุ เขตทุ่งครุ กรุงเทพมหานคร ประเทศไทย"

ticket_id: 2024-L23U9M
ประเภท: PM2.5
สถานที่: 70/136 ซอย 7/1 แขวงคันนายาว เขตคันนายาว กรุงเทพมหานคร 10230 ประเทศไทย
รายละเอียด: ชนิด: PM2.5 : "ในเวลานี้ เผาอีกแล้วค่ะ ถ้าบอกว่า ทำอาหาร ก็น่าจะ

In [ ]:
input_message = "ถ้าเกิดปัญหาทางเท้าเสียหายในเขตบางนา โมเดลจะสามารถหาข้อมูลที่เกี่ยวข้องได้หรือไม่?"

for step in graph.stream(
    {"messages": [{"role": "user", "content": input_message}]},
    stream_mode="values",
    config=config,
):
    step["messages"][-1].pretty_print()


================================ Human Message =================================

ถ้าเกิดปัญหาทางเท้าเสียหายในเขตบางนา โมเดลจะสามารถหาข้อมูลที่เกี่ยวข้องได้หรือไม่?
================================== Ai Message ==================================
Tool Calls:
  retrieve (2cbc51d9-b541-4468-96f8-6b5679e18060)
 Call ID: 2cbc51d9-b541-4468-96f8-6b5679e18060
  Args:
    query: damaged footpath in Bang Na district
================================= Tool Message =================================
Name: retrieve

ticket_id: 2024-6QEMGX
ประเภท: การเดินทาง, ร้องเรียน
สถานที่: ชุมชนหลักสี่พัฒนา 99 แขวงตลาดบางเขน เขตหลักสี่ กรุงเทพมหานคร 10210 ประเทศไทย
รายละเอียด: ฅนแก่ ฅนพิการจะได้ไม่ต้องไต่ฃึ้นบันไดกว่าฃั้นเพื่อไปฃึ้นรถไฟทางไกลบนทางยกระดับและรถไฟฟ้าสายสีแดงให้ลำบากเหมือนในวันนี้อีกต่อไปด้วยค่ะ" ที่อยู่ "ชุมชนหลักสี่พัฒนา แขวงตลาดบางเขน เขตหลักสี่ กรุงเทพมหานคร ประเทศไทย"

ticket_id: 2022-MLG89A
ประเภท: ถนน, น้ำท่วม
สถานที่: 211/78 Bang Na - Trat Road Bang Na Khet Bang Na Bangkok 10260 แขวง บางนา เ